<a href="https://colab.research.google.com/github/lssvirid/--212_Svyryd-Valeria/blob/main/%D0%A8%D1%80%D0%B8%D1%84%D1%82_%D0%A6%D0%B5%D0%B7%D0%B0%D1%80%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# --- BLOCK 1: CONFIGURATION ---
# Алфавіти
ALPHABET_EN = "abcdefghijklmnopqrstuvwxyz"
ALPHABET_UA = "абвгґдеєжзиіїйклмнопрстуфхцчшщьюя"

# Еталонні частоти (Англ)
FREQ_EN = {
    'a': 8.17, 'b': 1.29, 'c': 2.78, 'd': 4.25, 'e': 12.70, 'f': 2.23,
    'g': 2.02, 'h': 6.09, 'i': 6.97, 'j': 0.15, 'k': 0.77, 'l': 4.03,
    'm': 2.41, 'n': 6.75, 'o': 7.51, 'p': 1.93, 'q': 0.10, 'r': 5.99,
    's': 6.33, 't': 9.06, 'u': 2.76, 'v': 0.98, 'w': 2.36, 'x': 0.15,
    'y': 1.97, 'z': 0.07
}

# Еталонні частоти (Укр)
FREQ_UA = {
    'а': 8.34, 'б': 1.53, 'в': 5.50, 'г': 1.59, 'ґ': 0.01, 'д': 3.06,
    'е': 4.59, 'є': 0.39, 'ж': 0.71, 'з': 2.10, 'и': 6.00, 'і': 6.23,
    'ї': 0.84, 'й': 1.24, 'к': 3.48, 'л': 3.93, 'м': 3.02, 'н': 6.60,
    'о': 9.28, 'п': 2.29, 'р': 4.73, 'с': 4.07, 'т': 4.79, 'у': 3.38,
    'ф': 0.35, 'х': 1.17, 'ц': 1.16, 'ч': 1.44, 'ш': 0.71, 'щ': 0.32,
    'ь': 1.83, 'ю': 0.70, 'я': 2.16
}

In [10]:
# --- BLOCK 2: CAESAR CIPHER CLASS ---
class CaesarCipher:
    def __init__(self, alphabet: str):
        self.alphabet = alphabet
        self.char_to_index = {char: idx for idx, char in enumerate(alphabet)}
        self.index_to_char = {idx: char for idx, char in enumerate(alphabet)}
        self.modulus = len(alphabet)

    def _transform(self, text: str, shift: int) -> str:
        result = []
        for char in text:
            is_upper = char.isupper()
            char_lower = char.lower()

            if char_lower in self.char_to_index:
                idx = self.char_to_index[char_lower]
                new_idx = (idx + shift) % self.modulus
                new_char = self.index_to_char[new_idx]
                if is_upper: new_char = new_char.upper()
                result.append(new_char)
            else:
                result.append(char)
        return "".join(result)

    def encrypt(self, text: str, key: int) -> str:
        return self._transform(text, key)

    def decrypt(self, text: str, key: int) -> str:
        return self._transform(text, -key)

In [11]:
# --- BLOCK 3: ANALYSIS METRICS ---
from collections import Counter

def get_frequency_distribution(text: str, alphabet: str):
    counter = Counter(text.lower())
    total_chars = sum(counter[char] for char in alphabet if char in counter)

    if total_chars == 0:
        return {char: 0.0 for char in alphabet}

    distribution = {}
    for char in alphabet:
        count = counter.get(char, 0)
        distribution[char] = (count / total_chars) * 100
    return distribution

def calculate_chi_squared(text_dist, ref_dist, alphabet):
    chi_sq = 0.0
    for char in alphabet:
        observed = text_dist.get(char, 0)
        expected = ref_dist.get(char, 0)
        if expected > 0:
            chi_sq += ((observed - expected) ** 2) / expected
    return chi_sq

In [12]:
# --- BLOCK 4: MAIN EXECUTION ---
def hack_caesar(ciphertext, alphabet, ref_freqs):
    cipher = CaesarCipher(alphabet)
    best_shift = 0
    min_chi = float('inf')
    best_text = ""

    print(f"--- Аналіз варіантів ---")
    # Перебираємо всі можливі ключі
    for shift in range(len(alphabet)):
        decrypted = cipher.decrypt(ciphertext, shift)
        current_freqs = get_frequency_distribution(decrypted, alphabet)
        chi = calculate_chi_squared(current_freqs, ref_freqs, alphabet)

        # Виводимо топ-3 найкращих (опціонально, щоб бачити процес)
        if chi < min_chi:
            min_chi = chi
            best_shift = shift
            best_text = decrypted

    return best_shift, best_text

# ТЕСТ
caesar = CaesarCipher(ALPHABET_UA)
text = "Криптографія це наука про математичні методи забезпечення конфіденційності, цілісності і автентичності інформації."
key = 10

print(f"Оригінал: {text}")
encrypted = caesar.encrypt(text, key)
print(f"Шифртекст: {encrypted}\n")

found_key, result = hack_caesar(encrypted, ALPHABET_UA, FREQ_UA)

print(f"\n--- РЕЗУЛЬТАТ ЗЛОМУ ---")
print(f"Знайдений ключ: {found_key}")
print(f"Розшифровано: {result}")

Оригінал: Криптографія це наука про математичні методи забезпечення конфіденційності, цілісності і автентичності інформації.
Шифртекст: Фьрщяшйьибсз гм чиафи щьш циямциярґчс цмяшлр пиімпщмґмччз фшчбслмчгсучшюяс, гсхсючшюяс с иїямчярґчшюяс счбшьцигст.

--- Аналіз варіантів ---

--- РЕЗУЛЬТАТ ЗЛОМУ ---
Знайдений ключ: 10
Розшифровано: Криптографія це наука про математичні методи забезпечення конфіденційності, цілісності і автентичності інформації.
